In [5]:
import pyfesom2 as pf
import xarray as xr
import os
import pandas as pd
from collections import OrderedDict
from itertools import product
import matplotlib.pylab as plt
import numpy as np
from matplotlib import cm
%matplotlib inline

In [6]:
# parameters cell
meshpath = '/work/ollie/nkolduno/meshes/CORE2_fesom2/'
abg = [50, 15, -90]
variable = 'temp'
input_paths = ["/work/ollie/nkolduno/output_visc_control/",
               '/work/ollie/nkolduno/output_option_1/',
               ]
input_names = ['control', 'opt1']
years = range(1948, 2009)
reference_path = '/work/ollie/nkolduno/CLIMATOLOGY/core2_meanz/'
reference_name = 'clim'
reference_years = 1948
figsize=(10, 5)
levels = [-1, 1, 41]
maxdepth = 2000
units = r'$^\circ$C'
ofile = None
region = "Eurasian basin"

In [3]:
if input_names is None:
    input_names = []
    for run in input_paths:
        run = os.path.join(run, '')
        input_names.append(run.split('/')[-2])

In [4]:
mesh = pf.load_mesh(meshpath, abg=abg, 
                    usepickle=True, usejoblib=False)

/work/ollie/nkolduno/meshes/CORE2_fesom2/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /work/ollie/nkolduno/meshes/CORE2_fesom2/pickle_mesh_py3_fesom2


In [5]:
hofm = OrderedDict()
mask = pf.get_mask(mesh, region)
data_reference = pf.get_data(reference_path, variable, reference_years, mesh, how=None, silent=True, compute=False)
hofm_reference = pf.hovm_data(data_reference, mesh, mask = mask)
for exp_path, exp_name  in zip(input_paths, input_names):
    data = pf.get_data(exp_path, variable, years, mesh, how=None, compute=False, silent=True)
    title = exp_name+" - "+reference_name
    hofm[title] = {}
    data_difference= pf.hovm_data(data, mesh, mask = mask) - hofm_reference.data
    hofm[title]['data'] = data_difference
    if (data_difference.max() == data_difference.min() == 0):
        hofm[title]['nodiff'] = True
    else:
        hofm[title]['nodiff'] = False
    del data
    print(exp_name)

/home/ollie/nkolduno/miniconda3/envs/pyfesom2/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)
/home/ollie/nkolduno/miniconda3/envs/pyfesom2/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)


control


/home/ollie/nkolduno/miniconda3/envs/pyfesom2/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)


opt1


In [6]:
def data_to_plot(hofm):
    plot_data = []
    plot_names = []
    for key, value in hofm.items():
        if value['nodiff'] is False:
            plot_data.append(value['data'])
            plot_names.append(key)
                
    return plot_data, plot_names

In [7]:
plot_data, plot_names = data_to_plot(hofm)
if not plot_data:
    print('There is no difference between fields')
    identical = True
else:
    identical = False

if len(plot_data) == 1:
    plot_data = plot_data[0]
    plot_names = plot_names[0]

In [7]:
levels = np.linspace(levels[0], levels[1], levels[2])
plt.figure(figsize=(10, 5))
pf.plot_xyz(mesh, plot_data,
             levels = levels, maxdepth=maxdepth, cmap = cm.coolwarm, title=plot_names, )
if ofile is not None:
    plt.savefig(ofile)
    os.system(f'convert {ofile} -trim {ofile}_trimmed.png')
    os.system(f'mv {ofile}_trimmed.png {ofile}')

NameError: name 'mesh' is not defined

<Figure size 720x360 with 0 Axes>